In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time
import tqdm

# third-party imports
import numpy as np
import pandas as pd
import requests

# environment variables
from dotenv import load_dotenv
load_dotenv()
ANYDEALS_API_KEY = os.getenv("ANYDEAL_KEY")
# customisations - ensure tables show all columns
# pd.set_option("max_columns", 100)
print(ANYDEALS_API_KEY)

d0754faa36ea0d0d87c737908e13de1b56e857e5


In [2]:
# # merge all the game data into one dataframe
# temp_df = pd.DataFrame()
# for i in range(0, 3000,10):
#     if i == 0:
#         temp_df = pd.read_csv(f"textfiles/game_info/last_successful_batch_{i}_game_info_anydeal.csv", index_col=0)
#     else:
#         temp_df = pd.concat([temp_df, pd.read_csv(f"textfiles/game_info/last_successful_batch_{i}_game_info_anydeal.csv", index_col=0)])
# temp_df.to_csv('anydeal.csv')

In [3]:
# # merge all the game prices into one dataframe
# temp_df = pd.DataFrame()
# for i in range(0, 3000,10):
#     if i == 0:
#         temp_df = pd.read_csv(f"textfiles/price_history/last_successful_batch_{i}_price_history.csv", index_col=0)
#     else:
#         temp_df = pd.concat([temp_df, pd.read_csv(f"textfiles/price_history/last_successful_batch_{i}_price_history.csv", index_col=0)])

# temp_df.to_csv('anydeal_prices.csv')

In [55]:
df_game_info = pd.read_csv('anydeal.csv')
df_price_history = pd.read_csv('anydeal_prices.csv')
df_price_history.rename(columns={'appid':'id'}, inplace=True)


TypeError: dtype '<class 'list'>' not understood

In [36]:
df_game_info[df_game_info['id'] == '018d937f-1569-701c-bf2f-21163ded8f02']

,id,slug,title,type,mature,assets,earlyAccess,achievements,tradingCards,appid,tags,releaseDate,developers,publishers,reviews,stats,players,urls
299,018d937f-1569-701c-bf2f-21163ded8f02,total-war-shogun-2,Total War SHOGUN™ 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,True,201270.0,"['Strategy', 'Historical', 'Turn-Based Strateg...",2011-03-14,"[{'id': 466, 'name': 'Creative Assembly'}, {'i...","[{'id': 95, 'name': 'SEGA'}, {'id': 5464, 'nam...","[{'score': 90, 'source': 'Metascore', 'count':...","{'rank': 154, 'waitlisted': 1524, 'collected':...","{'recent': 0, 'day': 1, 'week': 2, 'peak': 122}",{'game': 'https://isthereanydeal.com/game/tota...
383,018d937f-1569-701c-bf2f-21163ded8f02,total-war-shogun-2,Total War SHOGUN™ 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,True,201270.0,"['Strategy', 'Historical', 'Turn-Based Strateg...",2011-03-14,"[{'id': 466, 'name': 'Creative Assembly'}, {'i...","[{'id': 95, 'name': 'SEGA'}, {'id': 5464, 'nam...","[{'score': 90, 'source': 'Metascore', 'count':...","{'rank': 154, 'waitlisted': 1524, 'collected':...","{'recent': 0, 'day': 1, 'week': 2, 'peak': 122}",{'game': 'https://isthereanydeal.com/game/tota...


In [34]:
df_game_info[df_game_info.duplicated('id')]

,id,slug,title,type,mature,assets,earlyAccess,achievements,tradingCards,appid,tags,releaseDate,developers,publishers,reviews,stats,players,urls
383,018d937f-1569-701c-bf2f-21163ded8f02,total-war-shogun-2,Total War SHOGUN™ 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,True,201270.0,"['Strategy', 'Historical', 'Turn-Based Strateg...",2011-03-14,"[{'id': 466, 'name': 'Creative Assembly'}, {'i...","[{'id': 95, 'name': 'SEGA'}, {'id': 5464, 'nam...","[{'score': 90, 'source': 'Metascore', 'count':...","{'rank': 154, 'waitlisted': 1524, 'collected':...","{'recent': 0, 'day': 1, 'week': 2, 'peak': 122}",{'game': 'https://isthereanydeal.com/game/tota...
1220,018d937f-269a-71ef-ae12-7e4d194b3514,poker-night-at-the-inventory,Poker Night at the Inventory,NaN,False,[],False,False,False,NaN,[],NaN,[],[],"[{'score': 71, 'source': 'Metascore', 'count':...","{'rank': 553, 'waitlisted': 381, 'collected': ...",NaN,{'game': 'https://isthereanydeal.com/game/poke...
1670,018d937e-fdec-7040-bb13-3d96b353c3c0,wasteland-2-directors-cut,Wasteland 2: Director's Cut,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,False,240760.0,"['RPG', 'Post-apocalyptic', 'Turn-Based', 'Str...",2014-09-19,"[{'id': 807, 'name': 'inXile Entertainment'}]","[{'id': 807, 'name': 'inXile Entertainment'}, ...","[{'score': 82, 'source': 'Steam', 'count': 460...","{'rank': 303, 'waitlisted': 5029, 'collected':...","{'recent': 15, 'day': 16, 'week': 19, 'peak': ...",{'game': 'https://isthereanydeal.com/game/wast...
1939,018d937f-0896-71b1-94d9-82725d1e320a,fallout-new-vegas,Fallout®: New Vegas™,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,False,22380.0,"['Open World', 'RPG', 'Post-apocalyptic', 'Sin...",2010-10-19,"[{'id': 656, 'name': 'Obsidian Entertainment'}]","[{'id': 18, 'name': 'Bethesda Softworks'}, {'i...","[{'score': 96, 'source': 'Steam', 'count': 170...","{'rank': 31, 'waitlisted': 4053, 'collected': ...","{'recent': 5289, 'day': 5289, 'week': 5732, 'p...",{'game': 'https://isthereanydeal.com/game/fall...
2453,018d937f-0a68-709b-bcc7-4d842e4fe130,space-hulk-deathwing-enhanced-edition,Space Hulk: Deathwing - Enhanced Edition,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,False,816090.0,"['Action', 'Warhammer 40K', 'Co-op', 'FPS', 'S...",2018-05-22,"[{'id': 1633, 'name': 'Streum On Studio'}]","[{'id': 106, 'name': 'Focus Entertainment'}, {...","[{'score': 71, 'source': 'Steam', 'count': 757...","{'rank': 3213, 'waitlisted': 2661, 'collected'...","{'recent': 53, 'day': 87, 'week': 89, 'peak': ...",{'game': 'https://isthereanydeal.com/game/spac...


In [33]:
df_price_history[df_price_history.duplicated('id')]

,id,price_history
383,018d937f-1569-701c-bf2f-21163ded8f02,"[{'timestamp': '2024-07-11T19:18:56+02:00', 's..."
1220,018d937f-269a-71ef-ae12-7e4d194b3514,"[{'timestamp': '2018-07-24T22:22:11+02:00', 's..."
1670,018d937e-fdec-7040-bb13-3d96b353c3c0,"[{'timestamp': '2024-07-11T19:19:14+02:00', 's..."
1939,018d937f-0896-71b1-94d9-82725d1e320a,"[{'timestamp': '2024-07-11T19:18:39+02:00', 's..."
2453,018d937f-0a68-709b-bcc7-4d842e4fe130,"[{'timestamp': '2024-07-25T19:16:14+02:00', 's..."


In [37]:
# remove duplicates before merging

df_game_info.drop_duplicates('id', inplace=True)
df_price_history.drop_duplicates('id', inplace=True)
# merge the two dataframes

df = pd.merge(df_game_info, df_price_history, on='id', how='inner')

In [40]:
df_without_history = df[df['price_history'] == '[]']
df_without_history.to_csv('anydeal_without_history.csv')

In [41]:
# remove rows without price history
df.drop(df[df['price_history'] == '[]'].index, inplace=True)

In [ ]:
# remove free games

In [42]:
df

,id,slug,title,type,mature,assets,earlyAccess,achievements,tradingCards,appid,tags,releaseDate,developers,publishers,reviews,stats,players,urls,price_history
0,018d937f-19a5-7057-bb6d-314d586e6dbc,dota-2,Dota 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,False,True,570.0,"['Free to Play', 'MOBA', 'Multiplayer', 'Strat...",2013-07-09,"[{'id': 5445, 'name': 'Valve'}]","[{'id': 5445, 'name': 'Valve'}]","[{'score': 81, 'source': 'Steam', 'count': 227...","{'rank': 645, 'waitlisted': 100, 'collected': ...","{'recent': 508811, 'day': 689869, 'week': 7566...",{'game': 'https://isthereanydeal.com/game/dota...,"[{'timestamp': '2024-02-11T01:47:46+01:00', 's..."
1,018d937f-7851-7004-b780-3f657a301f9a,counter-strike-2,Counter-Strike 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,False,True,730.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",2012-08-21,"[{'id': 5445, 'name': 'Valve'}]","[{'id': 5445, 'name': 'Valve'}]","[{'score': 87, 'source': 'Steam', 'count': 814...","{'rank': 519, 'waitlisted': 238, 'collected': ...","{'recent': 940515, 'day': 1268934, 'week': 134...",{'game': 'https://isthereanydeal.com/game/coun...,"[{'timestamp': '2024-02-11T01:47:46+01:00', 's..."
2,018d937f-57ce-723f-b7e7-7c7b5df93471,pubg-battlegrounds,PUBG: BATTLEGROUNDS,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,False,578080.0,"['Survival', 'Shooter', 'Battle Royale', 'Mult...",2017-12-21,"[{'id': 13154, 'name': 'KRAFTON, Inc.'}]","[{'id': 13154, 'name': 'KRAFTON, Inc.'}]","[{'score': 58, 'source': 'Steam', 'count': 239...","{'rank': 1682, 'waitlisted': 959, 'collected':...","{'recent': 372658, 'day': 613911, 'week': 6695...",{'game': 'https://isthereanydeal.com/game/pubg...,"[{'timestamp': '2024-02-11T03:51:23+01:00', 's..."
3,018d937f-50c1-7086-807c-e020c98c72b2,palworld,Palworld,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,False,1623730.0,"['Open World', 'Multiplayer', 'Survival', 'Cre...",2024-01-19,"[{'id': 29012, 'name': 'Pocketpair'}]","[{'id': 29012, 'name': 'Pocketpair'}]","[{'score': 93, 'source': 'Steam', 'count': 268...","{'rank': 415, 'waitlisted': 10630, 'collected'...","{'recent': 38763, 'day': 46359, 'week': 59456,...",{'game': 'https://isthereanydeal.com/game/palw...,"[{'timestamp': '2024-07-18T19:16:16+02:00', 's..."
4,018d937e-fde4-72ff-a7af-45e4955a8dd6,team-fortress-2,Team Fortress 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,True,440.0,"['Free to Play', 'Hero Shooter', 'Multiplayer'...",2007-10-10,"[{'id': 5445, 'name': 'Valve'}]","[{'id': 5445, 'name': 'Valve'}]","[{'score': 86, 'source': 'Steam', 'count': 111...","{'rank': 454, 'waitlisted': 18, 'collected': 2...","{'recent': 69106, 'day': 82557, 'week': 97322,...",{'game': 'https://isthereanydeal.com/game/team...,"[{'timestamp': '2024-02-11T01:47:46+01:00', 's..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2989,018d937e-f973-71db-988b-4b9d2c183d55,battlefleet-gothic-armada,Battlefleet Gothic: Armada,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,True,363680.0,"['Strategy', 'Warhammer 40K', 'Space', 'RTS', ...",2016-04-01,"[{'id': 910, 'name': 'Tindalos Interactive'}]","[{'id': 106, 'name': 'Focus Entertainment'}, {...","[{'score': 68, 'source': 'Steam', 'count': 479...","{'rank': 3333, 'waitlisted': 2815, 'collected'...","{'recent': 26, 'day': 47, 'week': 47, 'peak': ...",{'game': 'https://isthereanydeal.com/game/batt...,"[{'timestamp': '2024-07-25T19:16:13+02:00', 's..."
2990,018d96f1-a1a9-70ec-9c8a-06ef9eb86465,occupy-mars-prologue-2020,Occupy Mars: Prologue (2020),game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,False,False,1310520.0,"['Open World Survival Craft', 'Survival', 'Ope...",2020-08-20,"[{'id': 8870, 'name': 'Pyramid Games'}]","[{'id': 10811, 'name': 'Pyramid Games S.A.'}]","[{'score': 78, 'source': 'Steam', 'count': 153...","{'rank': 24532, 'waitlisted': 99, '

In [18]:
# df.drop_duplicates(subset=['id'], keep = 'first', inplace = True)

In [70]:
temp_ = df[df['id'] == '018d937f-0896-71b1-94d9-82725d1e320a']

In [117]:
first_available_price = ''
regular_price_reduced = False
price_reduced_date = ''
historical_low = ''
historical_low_date = ''
historical_high_discount = ''
historical_high_discount_date = ''
first_discount = ''
first_discount_date = ''
last_discount = ''
last_discount_date = ''
average_duration_between_discounts_12months = ''
average_duration_of_discounts_12months = ''
avg_price = ''
avg_discount = ''
next_discount = ''
price_decay_rate = ''

price_items = {
    'first_available_price': first_available_price,
    'regular_price_reduced': regular_price_reduced, # boolean
    'price_reduced_date': price_reduced_date, # null if regular_price_reduced is false
    # 'first_available_date': '', # we already have it in the game info
    'historical_low': historical_low,
    'historical_low_date': historical_low_date,
    'historical_high_discount': historical_high_discount,
    'historical_high_discount_date': historical_high_discount_date,
    'first_discount': first_discount,
    'first_discount_date': first_discount_date,
    'last_discount': last_discount,
    'last_discount_date': last_discount_date,
    'average_duration_between_discounts_12months': average_duration_between_discounts_12months,
    'average_duration_of_discounts_12months': average_duration_of_discounts_12months,
    'avg_price': avg_price,
    'avg_discount': avg_discount,

    

    # testing modules
    # time series analysis items
    'next_discount': next_discount,
    # price decay is the average second derivative of a moving average of the price
    # we take 12 months of data, and calculate the moving average of the price
    # then we calculate the second derivative of the moving average
    # finally we average the second derivative over the 12 months
    'price_decay_rate': price_decay_rate

}

# convert the price history from string to json
history = temp_['price_history'].iloc[0]
history = history.replace("'", '"')
history = json.loads(history)

# create the history dataframe
history = pd.DataFrame(history)

# convert the timestamp to datetime, keeping days only
history['timestamp'] = pd.to_datetime(history['timestamp'],utc=True)
history['timestamp'] = history['timestamp'].dt.date

# remove the shop column, as it is the same for all rows
history.drop(columns=['shop'], inplace=True)

# extract the price info
history['regular_price'] = history['deal'].apply(lambda x: x['regular']['amount'])
history['current_price'] = history['deal'].apply(lambda x: x['price']['amount'])
history['discount'] = history['deal'].apply(lambda x: x['cut'])

# remove the deal column
history.drop(columns=['deal'], inplace=True)

# sort the dataframe by date
history.sort_values(by='timestamp', inplace=True, ascending=True)

if history['regular_price'].unique().shape[0] > 1:
    regular_price_reduced = True
    price_reduced_date = ''
else:
    regular_price_reduced = False
    price_reduced_date = None

history

,timestamp,regular_price,current_price,discount
166,2012-10-29,19.99,9.99,50
165,2012-11-22,19.99,11.99,40
164,2013-07-11,9.99,5.99,40
163,2013-07-25,9.99,9.99,0
162,2013-08-01,9.99,7.49,25
...,...,...,...,...
4,2024-03-21,9.99,9.99,0
3,2024-04-11,9.99,2.49,75
2,2024-04-19,9.99,9.99,0
1,2024-06-27,9.99,4.99,50


In [118]:
# construct a dataframe with the every day price history based on a given date range

def get_price_history(df):
    start_date = df.iloc[0]['timestamp']
    end_date = df.iloc[-1]['timestamp']
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    price_history = pd.DataFrame()
    price_history['timestamp'] = date_range
    price_history['timestamp'] = price_history['timestamp'].dt.date
    price_history = price_history.merge(df, how='left', on='timestamp')
    price_history['regular_price'].ffill(inplace=True)
    price_history['current_price'].ffill(inplace=True)
    price_history['discount'].ffill(inplace=True)

    return price_history

get_price_history(history).to_csv('sample_history.csv')


In [99]:
# 12 months average price
def monthly_avg_price(year, month, df):
    monthprice = df[(df['timestamp'].dt.year == year) & (df['timestamp'].dt.month == month)]
    if monthprice.empty:
        return None
    else:
        current_index = monthprice.index[0]
        start_price = df['current_price'].iloc[current_index-1]
        return start_price

monthly_avg_price(2021, 11, history)


38

In [87]:
history['regular_price'].unique()

array([ 9.99, 19.99])

In [ ]:

price_items = {
    'first_available_price': first_available_price,
    'regular_price_reduced': '', # boolean
    'price_reduced_date': '', # null if regular_price_reduced is false
    # 'first_available_date': '', # we already have it in the game info
    'historical_low': '',
    'historical_low_date': '',
    'historical_high_discount': '',
    'historical_high_discount_date': '',
    'first_discount': '',
    'first_discount_date': '',
    'last_discount': '',
    'last_discount_date': '',
    'average_duration_between_discounts_12months': '',
    'average_duration_of_discounts_12months': '',
    'avg_price': '',
    'avg_discount': '',

    

    # testing modules
    # time series analysis items
    'next_discount': '',
    # price decay is the average second derivative of a moving average of the price
    # we take 12 months of data, and calculate the moving average of the price
    # then we calculate the second derivative of the moving average
    # finally we average the second derivative over the 12 months
    'price_decay_rate': '', 

}

# get price history for a game
